In [2]:
!pip install -U transformers
!pip install -U datasets
!pip install tensorboard
!pip install sentencepiece
!pip install -U accelerate
!pip install evaluate
!pip install rouge_score

  Using cached datasets-2.20.0-py3-none-any.whl.metadata (19 kB)
  Using cached pyarrow-16.1.0-cp310-cp310-manylinux_2_28_x86_64.whl.metadata (3.0 kB)
Using cached datasets-2.20.0-py3-none-any.whl (547 kB)
Using cached pyarrow-16.1.0-cp310-cp310-manylinux_2_28_x86_64.whl (40.8 MB)
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.19.2
    Uninstalling datasets-2.19.2:
      Successfully uninstalled datasets-2.19.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 24.4.1 requires cubinlinker, which is not installed.
cudf 24.4.1 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 24.4.1 requires ptxcompiler, which is not installed.
cuml 24.4.0 requires cupy-cuda11x>=12.

In [3]:
from datasets import load_from_disk, DatasetDict, load_dataset, Dataset
import re
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
import spacy

nltk.download('stopwords')
nltk.download('wordnet')

!unzip /usr/share/nltk_data/corpora/wordnet.zip -d /usr/share/nltk_data/corpora/

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
Archive:  /usr/share/nltk_data/corpora/wordnet.zip
   creating: /usr/share/nltk_data/corpora/wordnet/
  inflating: /usr/share/nltk_data/corpora/wordnet/lexnames  
  inflating: /usr/share/nltk_data/corpora/wordnet/data.verb  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.adv  
  inflating: /usr/share/nltk_data/corpora/wordnet/adv.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.verb  
  inflating: /usr/share/nltk_data/corpora/wordnet/cntlist.rev  
  inflating: /usr/share/nltk_data/corpora/wordnet/data.adj  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.adj  
  inflating: /usr/share/nltk_data/corpora/wordnet/LICENSE  
  inflating: /usr/share/nltk_data/corpora/wordnet/citation.bib  
  inflating: /usr/share/nltk_data/

In [4]:
# Load the dataset from disk
dataset = load_dataset('ccdv/pubmed-summarization', trust_remote_code=True)
train_dataset = dataset['train']
test_dataset = dataset['test']
eval_dataset = dataset['validation']

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [5]:
print(train_dataset)
print(test_dataset)
print(eval_dataset)

Dataset({
    features: ['article', 'abstract'],
    num_rows: 119924
})
Dataset({
    features: ['article', 'abstract'],
    num_rows: 6658
})
Dataset({
    features: ['article', 'abstract'],
    num_rows: 6633
})


In [6]:
# Define a function for text cleaning
def clean_text(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'\s+', ' ', text)  # Remove extra whitespace
    text = re.sub(r'\{.*?\}', '', text)  # Remove text in curly braces
    text = re.sub(r'\[.*?\]', '', text)  # Remove text in square brackets
    text = re.sub(r'https?://\S+|www\.\S+', '', text)  # Remove URLs
    text = re.sub(r'<.*?>', '', text)  # Remove HTML tags
    text = re.sub(r'\([^()]*\bfigure\b[^()]*\)', '', text)
    return text.strip()

# Apply the text cleaning function to the dataset
def apply_cleaning(examples):
    examples['article'] = [clean_text(article) for article in examples['article']]
    return examples

train_dataset = train_dataset.map(apply_cleaning, batched=True, num_proc=4)
test_dataset = test_dataset.map(apply_cleaning, batched=True, num_proc=4)
eval_dataset = eval_dataset.map(apply_cleaning, batched=True, num_proc=4)

Map (num_proc=4):   0%|          | 0/119924 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/6658 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/6633 [00:00<?, ? examples/s]

In [7]:
from nltk.stem import WordNetLemmatizer
from collections import Counter

# Initialize stopwords, lemmatizer, and counter
stop_words = set(stopwords.words('english'))
stop_words_dict = Counter(stop_words)
lemmatizer = WordNetLemmatizer()

# Function to remove stopwords and perform lemmatization
def remove_stopwords_and_lemmatize(text):
    words = nltk.word_tokenize(text)
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words_dict]
    return ' '.join(lemmatized_words)

# Function to apply the above processing to the dataset
def apply_processing(examples):
    examples['article'] = [remove_stopwords_and_lemmatize(article) for article in examples['article']]
    return examples

# Apply the processing to the datasets
train_dataset = train_dataset.map(apply_processing, batched=True, num_proc=4)
test_dataset = test_dataset.map(apply_processing, batched=True, num_proc=4)
eval_dataset = eval_dataset.map(apply_processing, batched=True, num_proc=4)

# Save the processed datasets



Map (num_proc=4):   0%|          | 0/119924 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/6658 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/6633 [00:00<?, ? examples/s]

In [8]:
from transformers import AutoTokenizer

checkpoint = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

In [9]:
import multiprocessing
multiprocessing.set_start_method('spawn', force=True)

In [10]:
# Function to convert text data into model inputs and targets
def preprocess_function(examples):
    global tokenizer
    inputs = [f"summarize: {article}" for article in examples['article']]
    model_inputs = tokenizer(
        inputs,
        max_length=512,
        truncation=True,
        padding='max_length'
    )

    # Set up the tokenizer for targets
    targets = [summary for summary in examples['abstract']]
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            targets,
            max_length=128,
            truncation=True,
            padding='max_length'
        )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


In [11]:
import torch
from transformers import T5ForConditionalGeneration, TrainingArguments, Trainer
MODEL = "t5-small"

model = T5ForConditionalGeneration.from_pretrained(MODEL)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model.to(device)

# Total parameters and trainable parameters.
total_params = sum(p.numel() for p in model.parameters())
print(f"{total_params:,} total parameters.")
total_trainable_params = sum(
    p.numel() for p in model.parameters() if p.requires_grad)
print(f"{total_trainable_params:,} training parameters.")

2024-06-24 07:38:10.332687: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-24 07:38:10.332805: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-24 07:38:10.498373: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

cuda
60,506,624 total parameters.
60,506,624 training parameters.


In [12]:
# Apply the function to the whole dataset
tokenized_valid = eval_dataset.map(
    preprocess_function,
    batched=True,
    num_proc=4
)

/opt/conda/lib/python3.10/site-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Map (num_proc=4):   0%|          | 0/6633 [00:00<?, ? examples/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:3946: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:3946: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword ar

In [13]:
tokenized_train = train_dataset.map(
    preprocess_function,
    batched=True,
    num_proc=4
)

Map (num_proc=4):   0%|          | 0/119924 [00:00<?, ? examples/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:3946: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:3946: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:3946: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your 

In [ ]:
tokenized_train.save_to_disk('tokenized_train_dataset')
tokenized_valid.save_to_disk('valid_train_dataset')

In [10]:
tokenized_train = load_from_disk('/kaggle/input/final-kdd-dataset-eval/tokenized_train_dataset')
tokenized_valid = load_from_disk('/kaggle/input/final-kdd-dataset-eval/valid_train_dataset')

In [14]:
import evaluate

rouge = evaluate.load("rouge")
 
def compute_metrics(eval_pred):
    predictions, labels = eval_pred.predictions[0], eval_pred.label_ids
 
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
 
    result = rouge.compute(
        predictions=decoded_preds,
        references=decoded_labels,
        use_stemmer=True,
        rouge_types=[
            'rouge1',
            'rouge2',
            'rougeL'
        ]
    )
 
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
 
    return {k: round(v, 4) for k, v in result.items()}

In [15]:
def preprocess_logits_for_metrics(logits, labels):
    """
    Original Trainer may have a memory leak.
    This is a workaround to avoid storing too many tensors that are not needed.
    """
    pred_ids = torch.argmax(logits[0], dim=-1)
    return pred_ids, labels

In [16]:
EPOCHS=3
BATCH_SIZE=8
OUT_DIR = 'results_t5base'

In [18]:
model.save_pretrained('./fine_tuned_model_final')
tokenizer.save_pretrained('./fine_tuned_model_final')

('./fine_tuned_model_final/tokenizer_config.json',
 './fine_tuned_model_final/special_tokens_map.json',
 './fine_tuned_model_final/spiece.model',
 './fine_tuned_model_final/added_tokens.json',
 './fine_tuned_model_final/tokenizer.json')

In [17]:
training_args = TrainingArguments(
    output_dir=OUT_DIR,
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir=OUT_DIR,
    logging_steps=10,
    evaluation_strategy='epoch',
    save_strategy='epoch',
    save_total_limit=2,
    report_to='tensorboard',
    learning_rate=0.0001,
    dataloader_num_workers=4,
    fp16=True,  # Enable mixed precision training
    dataloader_pin_memory=True  # Pin memory to speed up dataloader
)
 
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_valid,
    preprocess_logits_for_metrics=preprocess_logits_for_metrics,
    compute_metrics=compute_metrics
)
 
history = trainer.train()

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
2024-06-24 07:47:46.091988: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-24 07:47:46.092276: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-24 07:47:46.094409: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-24 07:47:46.098519: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting t

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Gen Len
1,2.426100,2.199274,0.501000,0.208100,0.433300,125.913300
2,2.283300,2.153730,0.507200,0.215300,0.440400,125.913300
3,2.324600,2.138589,0.509000,0.218000,0.442500,125.913300


2024-06-24 08:44:58.114387: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-24 08:44:58.114458: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-24 08:44:58.116625: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-24 08:44:58.119532: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-24 08:44:58.119684: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factor